## 07-1 다중 분류 신경망을 만듭니다

In [1]:
import numpy as np

In [ ]:
class MultiClassNetwork:

    def __init__(self, units = 10, batch_size = 32, learning_rate = 0.1, l1 = 0, l2 = 0):
        self.units = units  # 은닉층의 뉴런 개수
        self.batch_size = batch_size  # 배치 크기
        self.w1 = None  # 은닉층의 가중치
        self.b1 = None  # 은닉층의 절편
        self.w2 = None  # 출력층의 가중치
        self.b2 = None  # 출력층의 절편
        self.a1 = None  # 은닉층의 활성화 출력
        self.losses = []  # 훈련 손실
        self.val_losses = []  # 검증 손실
        self.lr = learning_rate  # 학습률
        self.l1 = l1  # L1 손실 하이퍼파라미터
        self.l2 = l2  # L2 손실 하이퍼파라미터

    def forpass(self, x):
        z1 = np.dot(x, self.w1) + self.b1  # 첫 번째 층의 선형 식 계산
        self.a1 = self.sigmoid(z1)  # 활성화 함수 , 시그모이드 적용 !
        z2 = np.dot(self.a1, self.w2) + self.b2  # 두 번째 츠으이 선형 식 계산

    def backprop(self, x, err):
        m = len(x)  # 샘플 개수
        # 출력층의 가중치와 절편에 대한 그래디언트를 계산
        w2_grad = np.dot(self.a1.T, err) / m
        b2_grad = np.sum(err) / m
        # 시그모이드 함수까지 그래디언트를 계산
        err_to_hidden = np.dot(err, self.w2.T) * self.a1 * (1 - self.a1)
        # 은닉층의 가중치와 절편에 대한 그래디언트를 계산
        w1_grad = np.dot(x.T, err_to_hidden) / m
        b1_grad = np.sum(err_to_hidden, axis = 0) / m
        return w1_grad, b1_grad, w2_grad, b2_grad

    def sigmoid(self, z):
        z = np.clip(z, -100, None)  # 안전한 계산위해 클리핑 
        a = 1 / (1 + np.exp(-z))
        return a

    def softmax(self, z):
        z = np.clip(z, -100, None)  # 안전한 np.exp() 계산 위해 클리핑
        exp_z = np.exp(z)
        return exp_z / np.sum(exp_z, axis = 1).reshape(-1, 1)

    def init_weights(self, n_features, n_classes):
        self.w1 = np.random.normal(0, 1, (n_features, self.units))  # (특성 개수, 은닉층의 크기)
        self.b1 = np.zeros(self.units)  # 은닉층의 크기
        self.w2 = np.random.normal(0, 1, (self.units, n_classes))  # (은닉층의 크기, 클래스 개수)
        self.b2 = np.zeros(n_classes)

    def fit(self, x, y, epochs = 100, x_val = None, y_val = None):
        np.random.seed(42)
        self.init_weights(x.shape[1], y.shape[1])
        for i in range(epochs):
            loss = 0
            print('.', end='')
            # 제너레이터 함수에서 반환한 미니배치를 순환한다.
            for x_batch, y_batch in self.gen_batch(x, y):
                a = self.training(x_batch, y_batch)
                # 안전한 계산 위해 클리핑
                a = np.clip(a, 1e-10, 1-1e-10)
                # 로그 손실과 규제 손실을 더해서 리스트에 추가
                loss += np.sum(-y_batch*np.log(a))
            self.losses.append((loss + self.reg_loss()) / len(X))
            # 검증 세트에 대한 손실을 계산한다.
            self.update_val_loss(x_val, y_val)

        def gen_batch(self, x, y):
            length = len(x)
            bins = length // self.batch_size
            if length % self.batch_size:
                bins += 1  # 나누어 떨어지지 않을 때
            indexes = np.random.permutation(np.arange(len(x))) # 인덱스를 섞습니다.
            x = x[indexes]
            y = y[indexes]
            for i in range(bins):
                start = self.batch_size * i
                end = self.batch_size * (i + 1)
                yield x[start:end], y[start:end]  # 배치 사이즈만큼 슬라이싱해서 반환

        def training(self, x, y):
            m = len(x)  # 샘플 개수를 저장
            z = self.forpass(x)
            a = self.softmax(z)
            err = -(y - a)
            # 오차를 역전파해서 그래디언트를 계산
            w1_grad, b1_grad, w2_grad, b2_grad = self.backprop(x, err)
            # 그래디언트에서 페널티 항의 미분 값을 뺀다.
            w1_grad += (self.l1 * np.sign(self.w1) + self.l2 * self.w1) / m
            w2_grad += (self.l1 * np.sign(self.w2) + self.l2 * self.w2) / m
            # 은닉층의 가중치와 절편을 업데이트합니다.
            self.w1 -= self.lr * w1_grad
            self.b1 -= self.lr * b1_grad
            # 출력층의 가중치와 절편을 업데이트합니다.
            self.w2 -= self.lr * w2_grad
            self.b2 -= self.lr * b2_grad
            return a

        def predict(self, x):
            z = self.forpass(x)
            return np.argmax(z, axis = 1)  # 가장 큰 값의 인덱스를 반환

        def score(self, x, y):
            # 예측과 타깃 열벡터를 비교해서 True의 비율을 반환
            return np.mean(self.predict(x) == np.argmax(y, axis = 1))

        def reg_loss(self):
            # 은닉층과 출력층의 가중치에 규제를 적용합니다.
            return self.l1 * (np.sum(np.abs(self.w1)) + np.sum(np.abs(self.w2))) + \
                self.l2 / 2 * (np.sum(self.w1**2) + np.sum(self.w2**2))

        def update_val_loss(self, x_val, y_val):
            z = self.forpass(x_val)
            a = self.softmax(z)
            a = np.clip(a, 1e-10, 1-1e-10)
            # 크로스 엔트로피 손실과 규제 손실을 더해서 리스트에 추가
            val_loss = np.sum(-y_val*np.log(a))
            self.val_losses.append((val_loss + self.reg_loss()) / len(y_val))